In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
my_zip = zipfile.ZipFile('/content/drive/MyDrive/양정현/net/Dataset.zip','r')
my_zip.extractall('./AI_Programming')

In [ ]:
import cv2
import os
import numpy as np
import zipfile
import torch

def image_load(img_path, height=128, width=128, bshort=False):
    # ---- image total number
    img_names = sorted(os.listdir(img_path))

    if not bshort:
        img_num = len(img_names)
    else:
        img_num = 10000

    print("img num : %d " % img_num)
    images = np.zeros((img_num, height, width, 3), dtype=np.uint8)

    for it in range(img_num):
        images[it, :, :, :] = cv2.imread(img_path + '%s' %(img_names[it]))

    return images

def gt_load(gt_file):
    f = open(gt_file)
    lines = f.readlines()
    num_gt = len(lines)
    print('gt num : %d ' %num_gt)

    gts = np.zeros(num_gt, dtype=np.int32)

    for it in range(num_gt):
        gts[it] = int(lines[it][:-1]) - 1  # saved gt values range : 1 ~ 200 -> 0 ~ 199

    f.close()

    return gts

def mini_batch_image(train_img, train_gts, batch_size, crop_size=96):
    batch_img = np.zeros((batch_size, crop_size, crop_size, 3), dtype=np.float32)
    batch_gts = np.zeros(batch_size, dtype=np.long)

    rand_num = np.random.randint(0, train_img.shape[0], batch_size)

    for it in range(batch_size):
        img = train_img[rand_num[it], :, :, :]

        # --- dataAug 1. Flip
        rand_flip = np.random.normal(0.0, scale=1.0)
        if rand_flip < 0:
            img = cv2.flip(img, 1)

        # --- dataAug 2. Crop
        crop_y = np.random.randint(0, img.shape[0] - crop_size - 1)
        crop_x = np.random.randint(0, img.shape[1] - crop_size - 1)
        img = img[crop_y: crop_y + crop_size, crop_x:crop_x + crop_size, :]

        batch_img[it, :, :, :] = (img / 255.0 * 2.0) - 1.0
        batch_gts[it] = train_gts[rand_num[it]]

    return batch_img, batch_gts

In [ ]:
import torch.nn as nn

class residual_block(nn.Module):
    def __init__(self, c_in,c_mid, c_out, bdown=False):
        super(residual_block, self).__init__()

        self.c_in = c_in
        self.c_mid = c_mid
        self.c_out = c_out

        self.bdown = bdown
        if self.bdown:
            stride = 2
        else:
            stride = 1

        self.convs = nn.Sequential(nn.BatchNorm2d(c_in),
                      nn.ReLU(),
                      nn.Conv2d(c_in, c_mid, kernel_size=(1, 1), padding=0, stride=(1, 1)),
                      nn.BatchNorm2d(c_mid),
                      nn.ReLU(),
                      nn.Conv2d(c_mid, c_mid, kernel_size=(5, 5), padding=2,stride = (1,1)),
                      nn.BatchNorm2d(c_mid),
                      nn.ReLU(),
                      nn.Conv2d(c_mid, c_out, kernel_size=(1, 1), padding=0,stride = (1,1)),
        )

        if self.c_in != c_out:
            self.conv_db = nn.Conv2d(c_in, c_out, kernel_size=(1, 1), stride=(1, 1),padding = 0)

    def forward(self, x):
        y = x
        x = self.convs(x)

        if self.c_in != self.c_out:
            y = self.conv_db(y)

        return (x + y) / np.sqrt(2.0)

class ResNet_50(nn.Module):
    def __init__(self, c_in=3, conv_ch=64, output_size=200):
        super(ResNet_50, self).__init__()
        self.ch = conv_ch

        self.conv = nn.Conv2d(c_in, self.ch, kernel_size=(7, 7), padding=3, stride=(2, 2))
        self.maxp = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        self.resblocks = nn.Sequential(residual_block(self.ch,self.ch, self.ch*4),
                                       residual_block(self.ch*4,self.ch, self.ch*4),
                                       residual_block(self.ch*4,self.ch, self.ch*4),

                                       residual_block(self.ch*4, self.ch*2, self.ch*8, bdown=True),
                                       residual_block(self.ch*8, self.ch*2,self.ch*8),
                                       residual_block(self.ch*8, self.ch*2,self.ch*8),
                                       residual_block(self.ch*8, self.ch*2,self.ch*8),


                                       residual_block(self.ch*8, self.ch*4,self.ch*16, bdown=True),
                                       residual_block(self.ch*16, self.ch*4,self.ch*16),
                                       residual_block(self.ch*16, self.ch*4,self.ch*16),
                                       residual_block(self.ch*16, self.ch*4,self.ch*16),
                                       residual_block(self.ch*16, self.ch*4,self.ch*16),
                                       residual_block(self.ch*16, self.ch*4,self.ch*16),

                                       residual_block(self.ch*16, self.ch*8,self.ch*32, bdown=True),
                                       residual_block(self.ch*32, self.ch*8, self.ch*32),
                                       residual_block(self.ch*32, self.ch*8, self.ch*32),

                                       nn.BatchNorm2d(self.ch*32),
                                       nn.ReLU(),
        )

        self.dr = nn.Dropout(p=0.2)
        self.fc = nn.Linear(32* self.ch, output_size)

    def forward(self, x):
        x = self.conv(x)
        x = self.maxp(x)
        x = self.resblocks(x)
        x = torch.mean(x, dim=(2, 3))
        x = self.dr(x)
        x = self.fc(x)

        return x

In [ ]:
import os, time

# 1. Paramters setting
num_class = 200
batch_size = 64  # or 32
initial_lr = 0.01  # 0.01 ---> 0
max_iter = 130000  # or 15~20만

save_name = 'Resnet50_SGD_b64_v2'

model_save_path = './model/%s/' %save_name
model_saving_iter = 5000
brestore = False
restore_iter = 6000
if not brestore:
    restore_iter = 0

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

# ----- 2. data load
print('DATA LOAD')
train_images = image_load("/content/AI_Programming/train/")
train_gts = gt_load("/content/AI_Programming/train_gt.txt")

test_images = image_load("/content/AI_Programming/test/", bshort=True)
test_gts = gt_load("/content/AI_Programming/test_gt_short.txt")

print('DATA LOAD FINISH')

# ---- 3. network build (restore model if necessary)
model = ResNet_50().to(DEVICE)

if brestore:
    model.load_state_dict(torch.load(model_save_path + 'model_%d.pt' % restore_iter))

# ---- 4. loss function and optimizer
loss = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr, eps=1.0)
optimizer = torch.optim.SGD(model.parameters(), lr=initial_lr, weight_decay=1e-4, momentum=0.9)

start_time = time.time()
for it in range(restore_iter, max_iter+1):
    optimizer.param_groups[0]['lr'] = initial_lr - (it / max_iter) * initial_lr

    batch_img, batch_gts = mini_batch_image(train_images, train_gts, batch_size)
    batch_img = np.transpose(batch_img, (0, 3, 1, 2))

    # ----- training step
    model.train()
    optimizer.zero_grad()
    pred = model(torch.from_numpy(batch_img).to(DEVICE))
    gt_tensor = torch.tensor(batch_gts, dtype=torch.long).to(DEVICE)

    train_loss = loss(pred, gt_tensor)
    train_loss.backward()
    optimizer.step()

    if it % 100 == 0:
        consume_time = time.time() - start_time
        print("iter : %d  lr : %.5f   loss : %.5f  time : %.4f " % (it, optimizer.param_groups[0]['lr'], train_loss.item(), consume_time))
        start_time = time.time()

    if it % model_saving_iter == 0 and it != 0:
        print('SAVING MODEL')
        if not os.path.isdir(model_save_path):
            os.makedirs(model_save_path)

        torch.save(model.state_dict(), model_save_path + 'model_%d.pt' % it)
        print('SAVING MODEL FINISH')

        print('START TEST')
        model.eval()
        t1_count = 0
        t5_count = 0
        for itest in range(10000):
            test_img = test_images[itest:itest+1, :, :, :].astype(np.float32)
            test_img = (test_img / 255.0 * 2.0) - 1.0

            test_img = np.transpose(test_img, (0, 3, 1, 2))
            with torch.no_grad():
                pred = model(torch.from_numpy(test_img).to(DEVICE))

            gt = test_gts[itest]

            pred = pred.cpu().numpy()  # [1, 200]
            pred = np.reshape(pred, num_class)

            # ----- top5 accuracy
            for ik in range(5):
                max_index = np.argmax(pred)
                if int(gt) == int(max_index):
                    t5_count += 1
                    # ----- top1 accuracy
                    if ik == 0:
                        t1_count += 1

                pred[max_index] = -9999

        print('top1 : %f    top5 : %f \n' %(t1_count / 10000 * 100, t5_count / 10000 * 100))
        f = open('%s.txt' %save_name, 'a+')
        f.write('top1 : %f    top5 : %f \n' %(t1_count / 10000 * 100, t5_count / 10000 * 100))
        f.close()

cuda
DATA LOAD
img num : 207005 
gt num : 207005 
img num : 10000 
gt num : 10000 
DATA LOAD FINISH


<ipython-input-3-df32df591bd6>:41: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  batch_gts = np.zeros(batch_size, dtype=np.long)


iter : 0  lr : 0.01000   loss : 5.33023  time : 0.8644 
iter : 100  lr : 0.00999   loss : 5.17091  time : 16.2492 
iter : 200  lr : 0.00998   loss : 5.17531  time : 16.2338 
iter : 300  lr : 0.00998   loss : 5.01193  time : 16.2458 
iter : 400  lr : 0.00997   loss : 4.94319  time : 16.2318 
iter : 500  lr : 0.00996   loss : 4.61053  time : 16.2536 
iter : 600  lr : 0.00995   loss : 4.80545  time : 16.2358 
iter : 700  lr : 0.00995   loss : 4.81506  time : 16.2468 
iter : 800  lr : 0.00994   loss : 4.71118  time : 16.2448 
iter : 900  lr : 0.00993   loss : 4.59723  time : 16.2327 
iter : 1000  lr : 0.00992   loss : 4.53103  time : 16.2352 
iter : 1100  lr : 0.00992   loss : 4.49592  time : 16.2409 
iter : 1200  lr : 0.00991   loss : 4.51274  time : 16.2301 
iter : 1300  lr : 0.00990   loss : 4.40963  time : 16.2516 
iter : 1400  lr : 0.00989   loss : 4.28420  time : 16.2385 
iter : 1500  lr : 0.00988   loss : 4.41777  time : 16.2231 
iter : 1600  lr : 0.00988   loss : 4.09897  time : 16

In [ ]:
import os, time

# 1. Paramters setting
num_class = 200
model_save_path = './model/Resnet50_SGD_b64_v2/'
restore_iter = 130000

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

# ----- 2. data load
print('DATA LOAD')
test_images = image_load("/content/AI_Programming/test/")
print('DATA LOAD FINISH')

# ---- 3. network build (restore model if necessary)
model = ResNet_50().to(DEVICE)
model.load_state_dict(torch.load(model_save_path + 'model_%d.pt' % restore_iter))

model.eval()

print('START TEST')
count = 0
for itest in range(test_images.shape[0]):
    if itest % 100 == 0:
        print('%d / %d ' %(itest, test_images.shape[0]))

    test_img = test_images[itest:itest+1, :, :, :].astype(np.float32)
    test_img = (test_img / 255.0 * 2.0) - 1.0

    test_img = np.transpose(test_img, (0, 3, 1, 2))
    with torch.no_grad():
        pred = model(torch.from_numpy(test_img).to(DEVICE))

    pred = pred.cpu().numpy()  # [1, 200]
    pred = np.reshape(pred, num_class)

    f = open('딥머닝_Resnet50_SGD_b64_v2.txt', 'a+')

    for ik in range(5):
        max_index = np.argmax(pred)
        f.write('%d ' % (int(max_index)))
        pred[max_index] = -9999

    f.write('\n')
    f.close()

cuda
DATA LOAD
img num : 51752 
DATA LOAD FINISH
START TEST
0 / 51752 
100 / 51752 
200 / 51752 
300 / 51752 
400 / 51752 
500 / 51752 
600 / 51752 
700 / 51752 
800 / 51752 
900 / 51752 
1000 / 51752 
1100 / 51752 
1200 / 51752 
1300 / 51752 
1400 / 51752 
1500 / 51752 
1600 / 51752 
1700 / 51752 
1800 / 51752 
1900 / 51752 
2000 / 51752 
2100 / 51752 
2200 / 51752 
2300 / 51752 
2400 / 51752 
2500 / 51752 
2600 / 51752 
2700 / 51752 
2800 / 51752 
2900 / 51752 
3000 / 51752 
3100 / 51752 
3200 / 51752 
3300 / 51752 
3400 / 51752 
3500 / 51752 
3600 / 51752 
3700 / 51752 
3800 / 51752 
3900 / 51752 
4000 / 51752 
4100 / 51752 
4200 / 51752 
4300 / 51752 
4400 / 51752 
4500 / 51752 
4600 / 51752 
4700 / 51752 
4800 / 51752 
4900 / 51752 
5000 / 51752 
5100 / 51752 
5200 / 51752 
5300 / 51752 
5400 / 51752 
5500 / 51752 
5600 / 51752 
5700 / 51752 
5800 / 51752 
5900 / 51752 
6000 / 51752 
6100 / 51752 
6200 / 51752 
6300 / 51752 
6400 / 51752 
6500 / 51752 
6600 / 51752 
6700 / 51752 
